**Problem (unicode1): Understanding Unicode (1 point)**


In [143]:
import regex as re  # Use regex module for Unicode property support
from dataclasses import dataclass
import json

from collections import Counter, defaultdict

In [5]:
"this is a test" + chr(0) + "string"

'this is a test\x00string'

In [6]:
print("this is a test" + chr(0) + "string")

this is a test string


In [7]:
test_string = "hello! こんにちは!"

In [8]:
utf8_encoded = test_string.encode("utf-8")

In [9]:
print(utf8_encoded)

b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'


In [10]:
print(type(utf8_encoded))

<class 'bytes'>


In [11]:
list(utf8_encoded)

[104,
 101,
 108,
 108,
 111,
 33,
 32,
 227,
 129,
 147,
 227,
 130,
 147,
 227,
 129,
 171,
 227,
 129,
 161,
 227,
 129,
 175,
 33]

In [12]:
print(len(test_string))

13


In [13]:
print(len(utf8_encoded))

23


In [14]:
print(utf8_encoded.decode("utf-8"))

hello! こんにちは!


In [141]:
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

In [16]:
re.findall(PAT, "some text that i'll pre-tokenize")

['some', ' text', ' that', ' i', "'ll", ' pre', '-', 'tokenize']

In [17]:
for _ in re.finditer(PAT, "some text that i'll pre-tokenize"):
    print(_.group())

some
 text
 that
 i
'll
 pre
-
tokenize


In [298]:
# read TinyStories dataset

with open("../data/TinyStoriesV2-GPT4-valid.txt", "r") as f:
    tiny_stories = f.readlines()
# tiny_stories


In [26]:
tiny_stories_example = tiny_stories[0][:120]

In [27]:
tiny_stories_example

'u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little girl. She was very k'

In [122]:
tokens = list(tiny_stories_example.encode("utf-8"))

for _ in tiny_stories_example[:10]:
    print(_, _.encode('utf-8')[0])

u 117
  32
d 100
o 111
n 110
' 39
t 116
  32
h 104
a 97


In [123]:
tokens[:10]

[117, 32, 100, 111, 110, 39, 116, 32, 104, 97]

In [124]:
def get_stats(ids):
    stats = {}
    for pair in zip(ids, ids[1:]):
        stats[pair] = stats.get(pair, 0)+1
    return stats

# stats = get_stats(tokens)

def merge(ids, pair, idx):
    newids = []
    i=0
    while i<len(ids):
        if ids[i] == pair[0] and ids[i+1]==pair[1]:
            newids.append(idx)
            i+=2
        else:
            newids.append(ids[i])
            i+=1
    return newids

In [125]:
merge([4, 5, 6, 7, 8, 9, 6, 7], [6,7], 999)

[4, 5, 999, 8, 9, 999]

In [126]:
# tokens2 = merge(tokens, top_pair, max(tokens)+1)

In [127]:
def train_bpe(tokens, iterations):
    merges = {}
    current_size = max(tokens)+1
    for i in range(iterations):
        print(f'Iterations - {i}')
        stats = get_stats(tokens)
        top_pair = max(stats, key = stats.get)
        print(f'top_pair - {top_pair}')
        print(f'Merging {top_pair} with new id - {current_size+i}')
        tokens = merge(tokens, top_pair, current_size+i)
        merges[top_pair] = current_size+i

    return tokens, merges

In [128]:
INIT_VOCAB_BYTES2IDS = {chr(id): id for id in range(255)}
INIT_VOCAB_IDS2BYTES = {id: chr(id) for id in range(255)}

In [44]:
with open("../data/TinyStoriesV2-GPT4-valid.txt", "r") as f:
    tsv = f.readlines()

tsv = tsv[:8]
tsv

['u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.\n',
 '<|endoftext|>\n',
 'Once upon a time, in a warm and sunny place, there was a big pit. A little boy named Tom liked to play near the pit. One day, Tom lost his red ball. He was very sad.\n',
 'Tom asked his friend, Sam, to help him search for the ball. They looked high and low, but they could not find the ball. Tom said, "I think my ball fell into the pit."\n',
 'Sam and Tom went close to the pit. They were scared, but they wanted to find the red ball. They looked into the pit, but it was too dark to see. Tom said, "We must go in and search for my ball."\n',
 'They went into the pit to search. It was dark and scary. They could not find the ball. They tried to get out, but the pit was too deep. Tom and Sam were stuck in the pit. They called

## Pre-tokenization

In [110]:
class BPETokenizer:
    def __init__(self):
        self.vocab = {} # token to id mapping i.e. b'0': 48
        self.inverse_vocab = {} # id to token mapping i.e. 48: b'0'
        self.merges = {} # dict of (pair): new_id merges
        self.special_tokens = {} # special tokens to id mapping
        pre_token_counts = {}

        self._init_vocab() # initialize vocabulary creation
        self.next_id = 256

        self.add_special_token('<|endoftext|>')
        self.PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

    def _init_vocab(self):
        for i in range(256):
            byte_token = bytes([i])
            self.vocab[byte_token] = i
            self.inverse_vocab[i] = byte_token

    def add_special_token(self, token_str):
        if token_str not in self.special_tokens:
            token_bytes = token_str.encode('utf-8')
            token_id = self.next_id

            self.special_tokens[token_bytes] = token_id

            self.vocab[token_bytes] = token_id
            self.inverse_vocab[token_id] = token_bytes

            self.next_id +=1
        
        return token_id
    
    def get_vocab_size(self):
        return len(self.vocab)
    
    def _bytes_to_tokens(self, text_bytes):
        return [bytes([b]) for b in text_bytes]
    
    def _tokens_to_bytes(self, tokens):
        return tokens.encode('utf-8')
    
    def tokenize(self, text):
        tokens = self.pre_tokenize(text)
        ids = [self.vocab.get(token, self.next_id) for token in tokens]
        return ids
    
    def detokenize(self, ids):
        tokens = [self.inverse_vocab.get(id, b'') for id in ids]
        return b''.join(tokens).decode('utf-8')

    def pre_tokenize(self, text, bytestring = True):
        # Tiny Stories text comes as a list of text strings including <|endoftext|>
        # function will turn this list of strings into list of tokens using PAT regex
        # we will use re.finditer to save memory
        stats = {}
        for i in range(len(text)):
            if text[i].rstrip().encode('utf-8') in self.vocab:
                token_bytes = text[i].rstrip().encode('utf-8')
                if bytestring:
                    key = token_bytes
                else:
                    key = tuple(token_bytes)  # Use tuple since lists aren't hashable
                stats[key] = stats.get(key, 0) + 1
            else:
                for match in re.finditer(PAT, text[i]):
                    token = match.group()
                    if token:
                        token_bytes = token.encode('utf-8')
                        if bytestring:
                            key = token_bytes
                        else:
                            key = tuple(token_bytes)  # Use tuple since lists aren't hashable
                        stats[key] = stats.get(key, 0) + 1

        return stats
    
    def merge(self, pair, idx):
        newids = []
        i = 0
        while i < len(self.vocab):
            if self.vocab[i] == pair[0] and self.vocab[i+1] == pair[1]:
                newids.append(idx)
                i += 2
            else:
                newids.append(self.vocab[i])
                i += 1
        return newids

    # def get_stats_pre_tokenizer(self):
    #     return self.stats


In [116]:
tokenizer = BPETokenizer()

# print(f"Initial vocab size: {tokenizer.get_vocab_size()}")

# # Test byte vocabulary
# test_text = "Дисципліновані aaa"
# test_bytes = tokenizer._tokens_to_bytes(test_text)
# print(f"Text '{test_text}' as bytes: {test_bytes}")
# print(f"Byte values: {list(test_bytes)}")

# # Convert to initial tokens
# initial_tokens = tokenizer._bytes_to_tokens(test_bytes)
# print(f"Initial tokens: {initial_tokens}")

# # Show token IDs
# token_ids = [tokenizer.vocab[token] for token in initial_tokens]
# print(f"Token IDs: {token_ids}")

# reconstructed = b''.join([tokenizer.inverse_vocab[tid] for tid in token_ids])
# print(f"Reconstructed bytes: {reconstructed}")
# print(f"Reconstructed text: {reconstructed.decode('utf-8')}")

In [115]:
tsv

['u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.\n',
 '<|endoftext|>\n',
 'Once upon a time, in a warm and sunny place, there was a big pit. A little boy named Tom liked to play near the pit. One day, Tom lost his red ball. He was very sad.\n',
 'Tom asked his friend, Sam, to help him search for the ball. They looked high and low, but they could not find the ball. Tom said, "I think my ball fell into the pit."\n',
 'Sam and Tom went close to the pit. They were scared, but they wanted to find the red ball. They looked into the pit, but it was too dark to see. Tom said, "We must go in and search for my ball."\n',
 'They went into the pit to search. It was dark and scary. They could not find the ball. They tried to get out, but the pit was too deep. Tom and Sam were stuck in the pit. They called

In [117]:
x = tokenizer.pre_tokenize(tsv, False)
x

{(117,): 1,
 (32, 100, 111, 110): 1,
 (39, 116): 1,
 (32, 104, 97, 118, 101): 1,
 (32, 116, 111): 9,
 (32, 98, 101): 1,
 (32, 115, 99, 97, 114, 101, 100): 3,
 (32, 111, 102): 2,
 (32, 116, 104, 101): 15,
 (32, 108, 111, 117, 100): 1,
 (32, 100, 111, 103): 1,
 (44,): 14,
 (32, 73): 1,
 (39, 108, 108): 1,
 (32, 112, 114, 111, 116, 101, 99, 116): 1,
 (32, 121, 111, 117): 1,
 (34, 46): 1,
 (32, 84, 104, 101): 2,
 (32, 109, 111, 108, 101): 3,
 (32, 102, 101, 108, 116): 1,
 (32, 115, 111): 1,
 (32, 115, 97, 102, 101): 2,
 (32, 119, 105, 116, 104): 1,
 (32, 108, 105, 116, 116, 108, 101): 2,
 (32, 103, 105, 114, 108): 1,
 (46,): 20,
 (32, 83, 104, 101): 1,
 (32, 119, 97, 115): 6,
 (32, 118, 101, 114, 121): 2,
 (32, 107, 105, 110, 100): 1,
 (32, 97, 110, 100): 10,
 (32, 115, 111, 111, 110): 1,
 (32, 99, 97, 109, 101): 1,
 (32, 116, 114, 117, 115, 116): 1,
 (32, 104, 101, 114): 2,
 (32, 72, 101): 2,
 (32, 108, 101, 97, 110, 101, 100): 1,
 (32, 97, 103, 97, 105, 110, 115, 116): 1,
 (32, 115, 104,

In [120]:
def get_stats_pairs(pre_tokenizer):
    pair_stats = {}
    for k in pre_tokenizer:
        if len(k)==1:
            pass
        elif len(k)==2:
            pair_stats[k] = pair_stats.get(k, 0) + 1
        else:
            for pair in zip(k, k[1:]):
                pair_stats[pair] = pair_stats.get(pair, 0)+1
    return pair_stats
        
get_stats_pairs(x)
    

{(32, 100): 5,
 (100, 111): 3,
 (111, 110): 4,
 (39, 116): 1,
 (32, 104): 8,
 (104, 97): 2,
 (97, 118): 1,
 (118, 101): 3,
 (32, 116): 10,
 (116, 111): 3,
 (32, 98): 6,
 (98, 101): 2,
 (32, 115): 12,
 (115, 99): 2,
 (99, 97): 4,
 (97, 114): 7,
 (114, 101): 4,
 (101, 100): 10,
 (32, 111): 3,
 (111, 102): 2,
 (116, 104): 6,
 (104, 101): 12,
 (32, 108): 7,
 (108, 111): 5,
 (111, 117): 5,
 (117, 100): 1,
 (111, 103): 1,
 (32, 73): 2,
 (39, 108): 1,
 (108, 108): 4,
 (32, 112): 4,
 (112, 114): 1,
 (114, 111): 1,
 (111, 116): 3,
 (116, 101): 3,
 (101, 99): 1,
 (99, 116): 1,
 (32, 121): 1,
 (121, 111): 1,
 (34, 46): 1,
 (32, 84): 3,
 (84, 104): 3,
 (32, 109): 3,
 (109, 111): 1,
 (111, 108): 1,
 (108, 101): 4,
 (32, 102): 6,
 (102, 101): 3,
 (101, 108): 3,
 (108, 116): 1,
 (115, 111): 2,
 (115, 97): 3,
 (97, 102): 1,
 (32, 119): 6,
 (119, 105): 1,
 (105, 116): 4,
 (108, 105): 2,
 (116, 116): 1,
 (116, 108): 1,
 (32, 103): 4,
 (103, 105): 1,
 (105, 114): 1,
 (114, 108): 1,
 (32, 83): 2,
 (83, 10

In [ ]:
@dataclass(frozen=True)
class BPETokenizer:
    def __init__(self):
        self.vocab = {} # token to id mapping i.e. b'0': 48
        self.inverse_vocab = {} # id to token mapping i.e. 48: b'0'
        self.merges = {} # dict of (pair): new_id merges
        self.special_tokens = {} # special tokens to id mapping
    
    def get_vocab_size(self):
        return len(self.vocab)
    
    def _bytes_to_tokens(self, text_bytes):
        return [bytes([b]) for b in text_bytes]
    
    def _tokens_to_bytes(self, tokens):
        return tokens.encode('utf-8')
    
    def tokenize(self, text):
        tokens = self.pre_tokenize(text)
        ids = [self.vocab.get(token, self.next_id) for token in tokens]
        return ids
    
    def detokenize(self, ids):
        tokens = [self.inverse_vocab.get(id, b'') for id in ids]
        return b''.join(tokens).decode('utf-8')
    
    def merge(self, pair, idx):
        newids = []
        i = 0
        while i < len(self.vocab):
            if self.vocab[i] == pair[0] and self.vocab[i+1] == pair[1]:
                newids.append(idx)
                i += 2
            else:
                newids.append(self.vocab[i])
                i += 1
        return newids

    # def get_stats_pre_tokenizer(self):
    #     return self.stats


In [ ]:
class BPETrainer:
    """
    Owns corpus scanning, pre-tokenisation, 
    Produces BPETokenizer
    """
    def __init__(self, bytestring = False):
        self.PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
        self.bytestring = bytestring
        self.special_tokens = {}
        self.vocab = {}
        self.inverse_vocab = {}

        # pre-populate initial_vocab
        self._init_vocab() # initialize vocabulary creation
        self.next_id = 256

        self.add_special_token('<|endoftext|>')

        #### Training variables
        self.wc = {}
        self.merges = {}
        self.segmented_words = {}
        self.pair_counts = {}
        

    def _init_vocab(self):
        for i in range(256):
            byte_token = bytes([i])
            self.vocab[byte_token] = i
            self.inverse_vocab[i] = byte_token

    def add_special_token(self, token_str):
        if token_str not in self.special_tokens:
            token_bytes = token_str.encode('utf-8')
            token_id = self.next_id

            self.special_tokens[token_bytes] = token_id

            self.vocab[token_bytes] = token_id
            self.inverse_vocab[token_id] = token_bytes

            self.next_id +=1
        
        return token_id
    

    def pre_tokenize(self, text):
        # Tiny Stories text comes as a list of text strings including <|endoftext|>
        # function will turn this list of strings into list of tokens using PAT regex
        # we will use re.finditer to save memory
        bytestring = self.bytestring
 
        for i in range(len(text)):
            if text[i].rstrip().encode('utf-8') in self.vocab:
                token_bytes = text[i].rstrip().encode('utf-8')
                if bytestring:
                    key = token_bytes
                else:
                    key = tuple([self.vocab.get(token_bytes)])  
                self.wc[key] = self.wc.get(key, 0) + 1
            else:
                for match in re.finditer(PAT, text[i]):
                    token = match.group()
                    if token:
                        token_bytes = token.encode('utf-8')
                        if bytestring:
                            key = token_bytes
                        else:
                            key = tuple(token_bytes)
                        self.wc[key] = self.wc.get(key, 0) + 1
                # Initialize each word as sequence of bytes (byte-level) or chars (unicode-level)
        
        self.segmented_words = {
            w: (list(w.encode("utf-8")) if self.byte_level else list(w.encode("utf-8")))  # placeholder; treat as bytes
            for w in wc
        }
        # Convert ints -> 1-byte bytes for consistency
        self.segmented_words = {w: [bytes([b]) for b in seq] for w, seq in self.segmented_words.items()}

        # Initialize vocab with all single bytes encountered (+ special tokens later)
        symbols = set(sym for seq in self.segmented_words.values() for sym in seq)
        self.vocab_index = {sym: i for i, sym in enumerate(sorted(symbols))}

        self._pairs_dirty = True

        return self
    
    def compute_pair_stats(self):
        self.pair_counts = self._compute_pair_counts()
        self._pairs_dirty = False

    def _compute_pair_counts(self):
        pair_counts = {}
        for w, freq in self.wc.items():
            if len(w)==1:
                pass
            elif len(w)==2:
                self.pair_counts[w] = freq # counts.get(w, 0) + freq
            else:
                for a, b in zip(w, w[1:]):
                    pair_counts[(a, b)] = pair_counts.get(tuple([a, b]),0) + freq

        return pair_counts


    
    def merge(self):

        tp = max(self.pair_counts, key = self.pair_counts.get)

        merged_symbol = self.inverse_vocab[tp[0]]+self.inverse_vocab[tp[1]]
        merged_id = len(self.vocab)

        self.vocab[merged_symbol] = merged_id
        self.inverse_vocab[merged_id] = merged_symbol

        for w, _ in self.wc:
            out = []
            i = 0

            while i<len(w):
                if i<len(w)-1 and w[i] == tp[0] and w[i+1] == tp[1]:
                    out.append(merged_id)
                    i+=2
                else:
                    out.append(w[i])

tokenizer = BPETokenizer()

trainer = BPETrainer()

trainer.pre_tokenize(tsv).compute_pair_stats()

In [257]:
x = trainer.pair_counts
v = trainer.vocab
iv = trainer.inverse_vocab
wc = trainer.wc

In [249]:
x

{(32, 100): 6,
 (100, 111): 2,
 (111, 110): 4,
 (32, 104): 13,
 (104, 97): 2,
 (97, 118): 1,
 (118, 101): 4,
 (32, 116): 35,
 (116, 111): 14,
 (32, 98): 16,
 (98, 101): 2,
 (32, 115): 19,
 (115, 99): 4,
 (99, 97): 6,
 (97, 114): 12,
 (114, 101): 9,
 (101, 100): 14,
 (32, 111): 5,
 (111, 102): 2,
 (116, 104): 22,
 (104, 101): 37,
 (32, 108): 9,
 (108, 111): 6,
 (111, 117): 8,
 (117, 100): 1,
 (111, 103): 1,
 (39, 108): 1,
 (108, 108): 10,
 (32, 112): 12,
 (112, 114): 1,
 (114, 111): 1,
 (111, 116): 4,
 (116, 101): 2,
 (101, 99): 1,
 (99, 116): 1,
 (32, 121): 1,
 (121, 111): 1,
 (32, 84): 15,
 (84, 104): 10,
 (32, 109): 6,
 (109, 111): 3,
 (111, 108): 3,
 (108, 101): 7,
 (32, 102): 11,
 (102, 101): 4,
 (101, 108): 4,
 (108, 116): 1,
 (115, 111): 2,
 (115, 97): 6,
 (97, 102): 2,
 (32, 119): 14,
 (119, 105): 1,
 (105, 116): 13,
 (108, 105): 3,
 (116, 116): 2,
 (116, 108): 2,
 (32, 103): 4,
 (103, 105): 1,
 (105, 114): 1,
 (114, 108): 1,
 (32, 83): 3,
 (83, 104): 1,
 (119, 97): 8,
 (97, 115

In [250]:
v

{b'\x00': 0,
 b'\x01': 1,
 b'\x02': 2,
 b'\x03': 3,
 b'\x04': 4,
 b'\x05': 5,
 b'\x06': 6,
 b'\x07': 7,
 b'\x08': 8,
 b'\t': 9,
 b'\n': 10,
 b'\x0b': 11,
 b'\x0c': 12,
 b'\r': 13,
 b'\x0e': 14,
 b'\x0f': 15,
 b'\x10': 16,
 b'\x11': 17,
 b'\x12': 18,
 b'\x13': 19,
 b'\x14': 20,
 b'\x15': 21,
 b'\x16': 22,
 b'\x17': 23,
 b'\x18': 24,
 b'\x19': 25,
 b'\x1a': 26,
 b'\x1b': 27,
 b'\x1c': 28,
 b'\x1d': 29,
 b'\x1e': 30,
 b'\x1f': 31,
 b' ': 32,
 b'!': 33,
 b'"': 34,
 b'#': 35,
 b'$': 36,
 b'%': 37,
 b'&': 38,
 b"'": 39,
 b'(': 40,
 b')': 41,
 b'*': 42,
 b'+': 43,
 b',': 44,
 b'-': 45,
 b'.': 46,
 b'/': 47,
 b'0': 48,
 b'1': 49,
 b'2': 50,
 b'3': 51,
 b'4': 52,
 b'5': 53,
 b'6': 54,
 b'7': 55,
 b'8': 56,
 b'9': 57,
 b':': 58,
 b';': 59,
 b'<': 60,
 b'=': 61,
 b'>': 62,
 b'?': 63,
 b'@': 64,
 b'A': 65,
 b'B': 66,
 b'C': 67,
 b'D': 68,
 b'E': 69,
 b'F': 70,
 b'G': 71,
 b'H': 72,
 b'I': 73,
 b'J': 74,
 b'K': 75,
 b'L': 76,
 b'M': 77,
 b'N': 78,
 b'O': 79,
 b'P': 80,
 b'Q': 81,
 b'R': 82,
 b'S': 

In [251]:
tp = trainer.find_best_pair()
tp

(104, 101)

In [303]:
from dataclasses import dataclass
from typing import Dict, List, Tuple, Iterable, Optional
import regex as re
import json
from collections import Counter, defaultdict

# ----------------------------
# Runtime Tokenizer (frozen)
# ----------------------------
@dataclass(frozen=True)
class BPETokenizer:
    merges: Tuple[Tuple[bytes, bytes], ...]              # ordered list of merges
    vocab: Tuple[bytes, ...]                             # index -> byte sequence
    special_tokens: Tuple[str, ...] = ()                 # e.g., ["<pad>", "<bos>", "<eos>", "<unk>"]
    byte_level: bool = False                              # byte-level BPE by default

    def encode(self, text: str) -> List[int]:
        """Apply merges to text and map to vocab IDs. No training state used."""
        # TODO: implement: normalize -> split -> bytes -> iterative merges -> ids
        raise NotImplementedError

    def decode(self, ids: List[int]) -> str:
        """Map ids back to byte sequences and join."""
        # TODO: implement reverse mapping: ids -> bytes -> text
        raise NotImplementedError

    def to_bytes(self) -> bytes:
        """Serialize tokenizer for saving to disk."""
        payload = {
            "merges": [[a.decode("latin1"), b.decode("latin1")] for a, b in self.merges],
            "vocab": [tok.decode("latin1") for tok in self.vocab],
            "special_tokens": list(self.special_tokens),
            "byte_level": self.byte_level,
        }
        return json.dumps(payload).encode("utf-8")

    @staticmethod
    def from_bytes(blob: bytes) -> "BPETokenizer":
        obj = json.loads(blob.decode("utf-8"))
        merges = tuple((a.encode("latin1"), b.encode("latin1")) for a, b in obj["merges"])
        vocab = tuple(s.encode("latin1") for s in obj["vocab"])
        return BPETokenizer(
            merges=merges,
            vocab=vocab,
            special_tokens=tuple(obj["special_tokens"]),
            byte_level=bool(obj["byte_level"]),
        )


# ----------------------------
# Trainer (stateful)
# ----------------------------
class BPETrainer:
    """
    Owns corpus scanning, pre-tokenization, stats, and merge learning.
    Produces a BPETokenizer at the end.
    """

    def __init__(
        self,
        # token_pattern: Optional[str] = r"\S+",
        # special_tokens: Optional[Iterable[str]] = None,
        byte_level: bool = True,
        # lowercase: bool = False,
    ):
        self.PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
        self.special_tokens = ['<|endoftext|>']
        self.byte_level = byte_level
        # self.lowercase = lowercase

        # Training state
        self.word_counts: Counter[str] = Counter()          # pretokenized words -> counts
        self.segmented_words: Dict[str, List[bytes]] = {}   # word -> current segmentation (list of byte “symbols”)
        self.pair_counts: Counter[Tuple[bytes, bytes]] = Counter()
        self.merges: List[Tuple[bytes, bytes]] = []
        self.vocab_index: Dict[bytes, int] = {}             # symbol -> id

        # Dirty flags
        self._pairs_dirty = True

    # ---------- Public pipeline ----------

    def pretokenize(self, corpus: Iterable[str]) -> "BPETrainer":
        """
        Scans the corpus once and stores word frequency stats.
        This is where your regex splitting lives.
        """
        bytestring = self.byte_level


        wc = Counter()
        for i in range(len(corpus)):
            if corpus[i].rstrip() in self.special_tokens:
                token_bytes = corpus[i].rstrip().encode('utf-8')
                # if bytestring:
                key = token_bytes
                # else:
                    # key = tuple([self.vocab.get(token_bytes)])  
                    # pass
                wc[key] = wc.get(key, 0) + 1
            else:
                for match in re.finditer(PAT, corpus[i]):
                    token = match.group()
                    if token:
                        token_bytes = token.encode('utf-8')
                        if corpus:
                            key = token_bytes
                        else:
                            key = tuple(token_bytes)
                        wc[key] = wc.get(key, 0) + 1

        self.word_counts = wc
        # Initialize each word as sequence of bytes (byte-level) or chars (unicode-level)
        self.segmented_words = {
            w: (list(w) if self.byte_level else list(w))  # placeholder; treat as bytes
            for w in wc
        }
        # Convert ints -> 1-byte bytes for consistency
        self.segmented_words = {w: [bytes([b]) for b in seq] for w, seq in self.segmented_words.items()}

        # Initialize vocab with all single bytes encountered (+ special tokens later)
        symbols = set(sym for seq in self.segmented_words.values() for sym in seq)
        self.vocab_index = {sym: i for i, sym in enumerate(sorted(symbols))}
        self._pairs_dirty = True
        return self

    def compute_pair_stats(self) -> "BPETrainer":
        """Builds pair frequency counts across all current segmentations."""
        self.pair_counts = self._compute_pair_counts()
        self._pairs_dirty = False
        return self

    def fit(self, n_merges: int, progress: bool = True) -> "BPETrainer":
        """Main training loop: repeatedly merge the most frequent pair and update state."""
        for i in range(n_merges):
            if self._pairs_dirty:
                self.compute_pair_stats()
            if not self.pair_counts:
                break
            best_pair, freq = self.pair_counts.most_common(1)[0]
            self._apply_merge(best_pair)
            self.merges.append(best_pair)
            self._pairs_dirty = True
            if progress and (i + 1) % max(1, n_merges // 10) == 0:
                # print(f"merge {i+1}/{n_merges}: {best_pair} ({freq})")
                pass
        return self

    def build_tokenizer(self) -> BPETokenizer:
        """Freeze training artifacts and return a runtime tokenizer."""
        # Build final vocab from current symbols + merged symbols + special tokens
        symbols = set(sym for seq in self.segmented_words.values() for sym in seq)
        # Keep existing indices if you want stable ids; here we rebuild simply
        vocab = tuple(sorted(symbols))
        # Optionally prepend special tokens as dedicated IDs; keep them as strings (runtime can map)
        return BPETokenizer(
            merges=tuple(self.merges),
            vocab=vocab,
            special_tokens=self.special_tokens,
            byte_level=self.byte_level,
        )

    # ---------- Private helpers (pure-ish) ----------

    def _compute_pair_counts(self) -> Counter:
        counts: Counter[Tuple[bytes, bytes]] = Counter()
        for w, freq in self.word_counts.items():
            seq = self.segmented_words[w]
            for a, b in zip(seq, seq[1:]):
                counts[(a, b)] += freq
        return counts

    def _apply_merge(self, pair: Tuple[bytes, bytes]) -> None:
        """
        Update all segmentations in-place by fusing occurrences of 'pair' into a single symbol.
        Also updates vocab_index incrementally.
        """
        merged_symbol = pair[0] + pair[1]  # concat bytes to represent the new symbol
        if merged_symbol not in self.vocab_index:
            self.vocab_index[merged_symbol] = len(self.vocab_index)

        # Incremental update of segmentations (touch only affected words)
        for w, seq in self.segmented_words.items():
            i = 0
            out: List[bytes] = []
            while i < len(seq):
                if i < len(seq) - 1 and seq[i] == pair[0] and seq[i + 1] == pair[1]:
                    out.append(merged_symbol)
                    i += 2
                else:
                    out.append(seq[i])
                    i += 1
            self.segmented_words[w] = out

        # After in-place updates, pair counts are now stale
        # We mark dirty and let compute_pair_stats rebuild, or implement a local delta update.
        # For simplicity in boilerplate: rebuild later.
        self._pairs_dirty = True

    # ---------- Optional utilities ----------

    def save_checkpoint(self, path: str) -> None:
        payload = {
            "word_counts": dict(self.word_counts),
            "merges": [[a.decode("latin1"), b.decode("latin1")] for a, b in self.merges],
            "special_tokens": list(self.special_tokens),
            "byte_level": self.byte_level,
            "lowercase": self.lowercase,
        }
        with open(path, "w", encoding="utf-8") as f:
            json.dump(payload, f)

    @staticmethod
    def load_checkpoint(path: str) -> "BPETrainer":
        with open(path, "r", encoding="utf-8") as f:
            obj = json.load(f)
        trainer = BPETrainer(
            special_tokens=obj["special_tokens"],
            byte_level=obj["byte_level"],
            lowercase=obj["lowercase"],
        )
        trainer.word_counts = Counter(obj["word_counts"])
        trainer.merges = [(a.encode("latin1"), b.encode("latin1")) for a, b in obj["merges"]]
        # segmented_words and pair_counts would need rebuild from word_counts and merges if desired
        return trainer


In [315]:
# tokenizer = BPETokenizer()

trainer = BPETrainer()

trainer.pretokenize(tiny_stories).compute_pair_stats().fit(5000)

In [316]:
trainer.word_counts

Counter({b'.': 421616,
         b',': 235432,
         b' the': 211031,
         b' and': 196057,
         b' a': 152161,
         b' to': 150493,
         b'\n': 122283,
         b' was': 108019,
         b' They': 52425,
         b' it': 51670,
         b' He': 49241,
         b' "': 47784,
         b' The': 46977,
         b' said': 43900,
         b' day': 43230,
         b' with': 42981,
         b' her': 38925,
         b' his': 38766,
         b' in': 38658,
         b' She': 38040,
         b' Tim': 37647,
         b' big': 35022,
         b' he': 32790,
         b' they': 29903,
         b' had': 28997,
         b' you': 28401,
         b'<|endoftext|>': 27630,
         b' not': 27019,
         b' happy': 25863,
         b' on': 25720,
         b' of': 25467,
         b' very': 25453,
         b' saw': 24931,
         b' that': 24693,
         b"'s": 24690,
         b' play': 24431,
         b' little': 23934,
         b' for': 21992,
         b' time': 21952,
         b' she'

In [317]:
trainer.merges

[(b' ', b't'),
 (b'h', b'e'),
 (b' ', b'a'),
 (b' ', b's'),
 (b' ', b'w'),
 (b'n', b'd'),
 (b' t', b'he'),
 (b'e', b'd'),
 (b' ', b'b'),
 (b' t', b'o'),
 (b' a', b'nd'),
 (b' ', b'h'),
 (b' ', b'f'),
 (b' ', b'T'),
 (b'i', b'n'),
 (b' w', b'a'),
 (b'r', b'e'),
 (b'i', b't'),
 (b'o', b'u'),
 (b' ', b'l'),
 (b' ', b'd'),
 (b' ', b'c'),
 (b' ', b'p'),
 (b'a', b'y'),
 (b' ', b'm'),
 (b'e', b'r'),
 (b' wa', b's'),
 (b' T', b'he'),
 (b'o', b'm'),
 (b' ', b'he'),
 (b'i', b's'),
 (b' ', b'n'),
 (b'i', b'm'),
 (b'a', b'r'),
 (b'o', b'n'),
 (b' s', b'a'),
 (b'l', b'l'),
 (b'i', b'd'),
 (b' h', b'a'),
 (b' ', b'g'),
 (b'a', b't'),
 (b' ', b'S'),
 (b'in', b'g'),
 (b'o', b't'),
 (b'e', b'n'),
 (b'a', b'n'),
 (b'l', b'e'),
 (b'o', b'r'),
 (b'e', b'nd'),
 (b'i', b'r'),
 (b'o', b'f'),
 (b'a', b'm'),
 (b'e', b't'),
 (b' ', b'H'),
 (b' ', b'it'),
 (b' t', b'h'),
 (b'i', b'g'),
 (b' The', b'y'),
 (b' p', b'l'),
 (b' ', b'in'),
 (b'i', b'l'),
 (b' H', b'e'),
 (b' ', b'"'),
 (b'o', b'w'),
 (b'v', b'er'),
 

In [318]:
tokenizer = trainer.build_tokenizer()

In [319]:
tokenizer.vocab

(b'\n',
 b' ',
 b' \n',
 b' "',
 b" '",
 b' -',
 b' 3',
 b' A',
 b' Abby',
 b' After',
 b' Al',
 b' Alex',
 b' Alice',
 b' All',
 b' Amy',
 b' And',
 b' Andy',
 b' Ann',
 b' Anna',
 b' Are',
 b' As',
 b' At',
 b' B',
 b' Be',
 b' Bear',
 b' Bee',
 b' Before',
 b' Bella',
 b' Ben',
 b' Benny',
 b' Bessie',
 b' Bet',
 b' Betty',
 b' Big',
 b' Bill',
 b' Billy',
 b' Binky',
 b' Bird',
 b' Birdy',
 b' Bl',
 b' Blue',
 b' Bob',
 b' Bobby',
 b' Bobo',
 b' Bop',
 b' Boun',
 b' Bounce',
 b' Bow',
 b' Br',
 b' Brown',
 b' Bu',
 b' Bubbles',
 b' Buddy',
 b' Bunny',
 b' But',
 b' Buzz',
 b' Buzzy',
 b' C',
 b' Can',
 b' Carl',
 b' Cat',
 b' Ch',
 b' Charlie',
 b' Che',
 b' Chirpy',
 b' Cl',
 b' Clara',
 b' Co',
 b' Coco',
 b' Come',
 b' D',
 b' Dad',
 b' Daddy',
 b' Daisy',
 b' Dan',
 b' Dave',
 b' Do',
 b' Dog',
 b' Doggy',
 b' Dolly',
 b' Don',
 b' Ducky',
 b' E',
 b' Ella',
 b' Ellie',
 b' Elly',
 b' Em',
 b' Emily',
 b' Emma',
 b' End',
 b' Even',
 b' Eventually',
 b' Every',
 b' Everyone',
 

In [358]:
class BPETrainer:
    """
    Owns corpus scanning, pre-tokenization, stats, and merge learning.
    Trains byte-level BPE and can export (vocab, merges).
    """

    def __init__(self, byte_level: bool = True):
        # GPT-style pattern; works with 'regex' (not 're')
        self.PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
        self.special_tokens: List[str] = ['<|endoftext|>']  # will be overwritten by wrapper
        self.byte_level = byte_level

        # Training state
        self.word_counts: Counter[bytes] = Counter()        # token (bytes) -> count
        self.segmented_words: Dict[bytes, List[bytes]] = {} # token (bytes) -> list of byte-symbols
        self.pair_counts: Counter[Tuple[bytes, bytes]] = Counter()
        self.merges: List[Tuple[bytes, bytes]] = []
        self.vocab_index: Dict[bytes, int] = {}             # symbol -> stable id (non-special)

        # Dirty flag
        self._pairs_dirty = True

    # ---------- Public pipeline ----------

    def pretokenize(self, corpus: Iterable[str]) -> "BPETrainer":
        """
        Scans the corpus once and stores token frequency stats.
        Special tokens are ignored for training (they don't affect merges).
        """
        wc: Counter[bytes] = Counter()
        specials = set(self.special_tokens)

        for line in corpus:
            line = line.rstrip("\n")
            # skip lines that are exactly a special token
            if line in specials:
                continue
            for m in re.finditer(self.PAT, line):
                tok = m.group(0)
                if not tok or tok in specials:
                    continue
                tok_bytes = tok.encode("utf-8")
                wc[tok_bytes] += 1

        self.word_counts = wc

        # Initialize segmentation: each token -> list of single-byte symbols
        self.segmented_words = {
            w: [bytes([b]) for b in w]  # w is already bytes; iterating yields ints
            for w in wc
        }

        # Initial vocab = all unique single-byte symbols observed
        base_symbols = [bytes([b]) for b in range(256)]
        self.vocab_index = {sym: i for i, sym in enumerate(base_symbols)}

        self._pairs_dirty = True
        return self

    def compute_pair_stats(self) -> "BPETrainer":
        """Rebuild pair frequency counts across current segmentations."""
        self.pair_counts = self._compute_pair_counts()
        self._pairs_dirty = False
        return self

    def fit_to_vocab_size(self, vocab_size: int, special_tokens: List[str], progress: bool = True) -> "BPETrainer":
        """
        Greedy BPE loop that stops when len(non-special symbols) + len(special_tokens) reaches vocab_size.
        """
        target_non_special = max(0, vocab_size - len(special_tokens))
        while len(self.vocab_index) < target_non_special:
            if self._pairs_dirty:
                self.compute_pair_stats()
            if not self.pair_counts:  # nothing left to merge
                break
            best_pair, _ = self.pair_counts.most_common(1)[0]
            self._apply_merge(best_pair)
            self.merges.append(best_pair)
            self._pairs_dirty = True
            if progress and (len(self.merges) % 1000 == 0):
                pass
        return self

    def export_vocab_and_merges(
        self,
        special_tokens: List[str],
        vocab_size: int
    ) -> Tuple[Dict[int, bytes], List[Tuple[bytes, bytes]]]:
        """
        Build final vocab (IDs -> bytes) with special tokens first, then symbols by stable index.
        Caps size at vocab_size.
        """
        vocab: Dict[int, bytes] = {i: s.encode("utf-8") for i, s in enumerate(special_tokens)}
        offset = len(special_tokens)

        # Non-special symbols in stable order
        items = sorted(self.vocab_index.items(), key=lambda kv: kv[1])
        for sym, idx in items:
            tok_id = offset + idx
            if tok_id >= vocab_size:
                break
            vocab[tok_id] = sym

        return vocab, list(self.merges)

    # ---------- Private helpers ----------

    def _compute_pair_counts(self) -> Counter:
        counts: Counter[Tuple[bytes, bytes]] = Counter()
        for w, freq in self.word_counts.items():
            seq = self.segmented_words[w]
            for a, b in zip(seq, seq[1:]):
                counts[(a, b)] += freq
        return counts

    def _apply_merge(self, pair: Tuple[bytes, bytes]) -> None:
        """
        Fuse occurrences of 'pair' into a single symbol across all segmentations.
        Update vocab_index with the new merged symbol if needed.
        """
        merged_symbol = pair[0] + pair[1]
        if merged_symbol not in self.vocab_index:
            self.vocab_index[merged_symbol] = len(self.vocab_index)

        for w, seq in self.segmented_words.items():
            i = 0
            out: List[bytes] = []
            while i < len(seq):
                if i < len(seq) - 1 and seq[i] == pair[0] and seq[i + 1] == pair[1]:
                    out.append(merged_symbol)
                    i += 2
                else:
                    out.append(seq[i])
                    i += 1
            self.segmented_words[w] = out

        self._pairs_dirty = True

    # ---------- Optional utilities ----------

    def save_checkpoint(self, path: str) -> None:
        payload = {
            "word_counts": {k.decode("latin1"): v for k, v in self.word_counts.items()},
            "merges": [[a.decode("latin1"), b.decode("latin1")] for a, b in self.merges],
            "special_tokens": list(self.special_tokens),
            "byte_level": self.byte_level,
        }
        with open(path, "w", encoding="utf-8") as f:
            json.dump(payload, f)

    @staticmethod
    def load_checkpoint(path: str) -> "BPETrainer":
        with open(path, "r", encoding="utf-8") as f:
            obj = json.load(f)
        t = BPETrainer(byte_level=obj["byte_level"])
        t.special_tokens = obj["special_tokens"]
        t.word_counts = Counter({k.encode("latin1"): v for k, v in obj["word_counts"].items()})
        t.merges = [(a.encode("latin1"), b.encode("latin1")) for a, b in obj["merges"]]
        # segmented_words and pair_counts would need rebuild from word_counts and merges if desired
        return t


# -------- Public training function (deliverable) --------

def train_bpe_tokenizer(input_path: str, vocab_size: int, special_tokens: List[str]):
    required_min = len(special_tokens) + 256
    if vocab_size < required_min:
        raise ValueError(
            f"vocab_size must be at least {required_min} "
            f"(= {len(special_tokens)} specials + 256 base bytes)."
        )

    trainer = BPETrainer(byte_level=True)
    trainer.special_tokens = list(special_tokens)  # ensures pretokenize ignores them

    with open(input_path, "r", encoding="utf-8") as f:
        trainer.pretokenize(f).compute_pair_stats().fit_to_vocab_size(vocab_size, special_tokens)

    vocab, merges = trainer.export_vocab_and_merges(special_tokens, vocab_size)
    return vocab, merges

In [359]:
vocab, merges = train_bpe_tokenizer(
    input_path="../data/TinyStoriesV2-GPT4-train.txt",
    vocab_size=100,
    special_tokens=["<|endoftext|>"],
)
print(f"Vocab size (returned): {len(vocab)} | Merges learned: {len(merges)}")

ValueError: vocab_size must be at least 257 (= 1 specials + 256 base bytes).

In [356]:
vocab

{0: b'<|endoftext|>',
 1: b'\x00',
 2: b'\x01',
 3: b'\x02',
 4: b'\x03',
 5: b'\x04',
 6: b'\x05',
 7: b'\x06',
 8: b'\x07',
 9: b'\x08',
 10: b'\t',
 11: b'\n',
 12: b'\x0b',
 13: b'\x0c',
 14: b'\r',
 15: b'\x0e',
 16: b'\x0f',
 17: b'\x10',
 18: b'\x11',
 19: b'\x12',
 20: b'\x13',
 21: b'\x14',
 22: b'\x15',
 23: b'\x16',
 24: b'\x17',
 25: b'\x18',
 26: b'\x19',
 27: b'\x1a',
 28: b'\x1b',
 29: b'\x1c',
 30: b'\x1d',
 31: b'\x1e',
 32: b'\x1f',
 33: b' ',
 34: b'!',
 35: b'"',
 36: b'#',
 37: b'$',
 38: b'%',
 39: b'&',
 40: b"'",
 41: b'(',
 42: b')',
 43: b'*',
 44: b'+',
 45: b',',
 46: b'-',
 47: b'.',
 48: b'/',
 49: b'0',
 50: b'1',
 51: b'2',
 52: b'3',
 53: b'4',
 54: b'5',
 55: b'6',
 56: b'7',
 57: b'8',
 58: b'9',
 59: b':',
 60: b';',
 61: b'<',
 62: b'=',
 63: b'>',
 64: b'?',
 65: b'@',
 66: b'A',
 67: b'B',
 68: b'C',
 69: b'D',
 70: b'E',
 71: b'F',
 72: b'G',
 73: b'H',
 74: b'I',
 75: b'J',
 76: b'K',
 77: b'L',
 78: b'M',
 79: b'N',
 80: b'O',
 81: b'P',
 82: b

In [357]:
merges

[(b' ', b't'),
 (b'h', b'e'),
 (b' ', b'a'),
 (b' ', b's'),
 (b' ', b'w'),
 (b' t', b'he'),
 (b'n', b'd'),
 (b'e', b'd'),
 (b' ', b'b'),
 (b' t', b'o'),
 (b' a', b'nd'),
 (b' ', b'h'),
 (b' ', b'f'),
 (b' ', b'T'),
 (b'i', b'n'),
 (b' w', b'a'),
 (b'r', b'e'),
 (b'i', b't'),
 (b'o', b'u'),
 (b' ', b'l'),
 (b' ', b'd'),
 (b' ', b'c'),
 (b' ', b'p'),
 (b'a', b'y'),
 (b' ', b'm'),
 (b'e', b'r'),
 (b' wa', b's'),
 (b' T', b'he'),
 (b'o', b'm'),
 (b' ', b'he'),
 (b'i', b's'),
 (b' ', b'n'),
 (b'i', b'm'),
 (b'a', b'r'),
 (b'o', b'n'),
 (b' s', b'a'),
 (b'l', b'l'),
 (b'i', b'd'),
 (b' h', b'a'),
 (b' ', b'g'),
 (b'a', b't'),
 (b' ', b'S'),
 (b'in', b'g'),
 (b'o', b't'),
 (b'e', b'n'),
 (b'a', b'n'),
 (b'l', b'e'),
 (b'o', b'r'),
 (b'i', b'r'),
 (b'a', b'm'),
 (b'e', b't'),
 (b' ', b'H'),
 (b' ', b'it'),
 (b' t', b'h'),
 (b'i', b'g'),
 (b' The', b'y'),
 (b' p', b'l'),
 (b' ', b'in'),
 (b'i', b'l'),
 (b' H', b'e'),
 (b' ', b'"'),
 (b'o', b'w'),
 (b'v', b'er'),
 (b'r', b'i'),
 (b' ', b'u'),
 (

In [350]:
from tests.common import FIXTURES_PATH, gpt2_bytes_to_unicode

In [351]:
# Path to the reference tokenizer vocab and merges
reference_vocab_path = FIXTURES_PATH / "train-bpe-reference-vocab.json"
reference_merges_path = FIXTURES_PATH / "train-bpe-reference-merges.txt"

In [353]:


# Compare the learned merges to the expected output merges
gpt2_byte_decoder = {v: k for k, v in gpt2_bytes_to_unicode().items()}
with open(reference_merges_path, encoding="utf-8") as f:
    gpt2_reference_merges = [tuple(line.rstrip().split(" ")) for line in f]
    reference_merges = [
        (
            bytes([gpt2_byte_decoder[token] for token in merge_token_1]),
            bytes([gpt2_byte_decoder[token] for token in merge_token_2]),
        )
        for merge_token_1, merge_token_2 in gpt2_reference_merges
    ]
# assert merges == reference_merges

# Compare the vocab to the expected output vocab
with open(reference_vocab_path, encoding="utf-8") as f:
    gpt2_reference_vocab = json.load(f)
    reference_vocab = {
        gpt2_vocab_index: bytes([gpt2_byte_decoder[token] for token in gpt2_vocab_item])
        for gpt2_vocab_item, gpt2_vocab_index in gpt2_reference_vocab.items()}

In [354]:
reference_merges

[(b' ', b't'),
 (b' ', b'a'),
 (b'h', b'e'),
 (b'i', b'n'),
 (b' t', b'he'),
 (b'r', b'e'),
 (b' ', b'o'),
 (b' ', b','),
 (b'e', b'r'),
 (b' ', b's'),
 (b'a', b't'),
 (b' ', b'.'),
 (b'n', b'd'),
 (b'i', b's'),
 (b'o', b'r'),
 (b' ', b'w'),
 (b' ', b'c'),
 (b'o', b'n'),
 (b' ', b'b'),
 (b' ', b'f'),
 (b'o', b'u'),
 (b'i', b't'),
 (b'e', b'n'),
 (b'e', b's'),
 (b' o', b'f'),
 (b' ', b'p'),
 (b'in', b'g'),
 (b' ', b'in'),
 (b'e', b'd'),
 (b'a', b'l'),
 (b' ', b'm'),
 (b' a', b'nd'),
 (b' ', b'd'),
 (b'a', b'n'),
 (b'a', b'r'),
 (b' t', b'o'),
 (b'o', b'm'),
 (b' t', b'h'),
 (b'i', b'c'),
 (b'i', b'on'),
 (b' ', b'h'),
 (b' ', b'l'),
 (b' ', b'y'),
 (b' ', b'e'),
 (b'a', b's'),
 (b'o', b't'),
 (b'i', b'l'),
 (b' ', b'n'),
 (b' ', b'u'),
 (b'en', b't'),
 (b' b', b'e'),
 (b' ', b'&'),
 (b' ', b'is'),
 (b' y', b'ou'),
 (b'o', b's'),
 (b' ', b're'),
 (b'e', b't'),
 (b' f', b'or'),
 (b'u', b't'),
 (b'e', b'l'),
 (b' ', b'g'),
 (b'a', b'y'),
 (b's', b't'),
 (b'o', b'w'),
 (b'l', b'e'),
 (b'c',